In [1]:
import io
import sqlite3
import pandas as pd

import matplotlib as plt
import csv

In [2]:
csvfile = open('2014106384.csv', 'w')
csvwriter = csv.writer(csvfile, delimiter='|',  quoting=csv.QUOTE_MINIMAL)

conn = sqlite3.connect('tp1.db')
cursor = conn.cursor()

f = io.open('despesas_publicas_tp1.sql', 'r', encoding='ISO-8859-1')
sql = f.read()
cursor.executescript(sql)

# Questões: 1 2   4 5   7 9

#### 1. Liste todos os códigos e nomes de subfunções da função que possui o maior número de subfunções.

In [3]:
consulta_questao1 = "SELECT A.COD_SUBFUNCAO, A.NOME_SUBFUNCAO FROM SUBFUNCAO AS A INNER JOIN (SELECT COD_SUBFUNCAO FROM DETALHAMENTO_FUNCAO WHERE COD_FUNCAO = (SELECT COD_FUNCAO FROM (SELECT COD_FUNCAO, MAX(Y.SUB) FROM ( SELECT COD_FUNCAO, COUNT(COD_SUBFUNCAO) as SUB FROM DETALHAMENTO_FUNCAO GROUP BY COD_FUNCAO ) Y ))) AS X ON A.COD_SUBFUNCAO = X.COD_SUBFUNCAO"
df = pd.read_sql_query(consulta_questao1, conn)
print(df)
csvwriter.writerow([1, df.shape[0], df.shape[1], consulta_questao1])

   COD_SUBFUNCAO               NOME_SUBFUNCAO
0            125  Normatização e fiscalização
1            126     Tecnologia da informação
2            131           Comunicação social
3            244      Assistência comunitária
4            306       Alimentação e nutrição
5            365            Educação infantil
6            511      Saneamento básico rural


349

#### 2. Liste o valor total de despesas públicas por função (nome) e subfunção(nome), apenas nos casos em que esse valor total excedeu 200 mil reais.

In [4]:
consulta_questao2 = "SELECT FUNC.NOME_FUNCAO, SUBFUNC.NOME_SUBFUNCAO, CODS.VALOR_DESPESA FROM FUNCAO AS FUNC, SUBFUNCAO AS SUBFUNC INNER JOIN (SELECT * FROM DETALHAMENTO_FUNCAO AS DETALHAMENTO INNER JOIN (SELECT * FROM DESPESA_PUBLICA WHERE VALOR_DESPESA > 200000) AS DESPESA ON DETALHAMENTO.COD_DETALHAMENTO = DESPESA.COD_DETALHAMENTO) AS CODS ON FUNC.COD_FUNCAO = CODS.COD_FUNCAO AND SUBFUNC.COD_SUBFUNCAO = CODS.COD_SUBFUNCAO"
df = pd.read_sql_query(consulta_questao2, conn)
print(df)
csvwriter.writerow([2, df.shape[0], df.shape[1], consulta_questao2])

          NOME_FUNCAO           NOME_SUBFUNCAO  VALOR_DESPESA
0  Assistência social  Assistência comunitária      219668.07
1            Educação          Educação básica      334126.70


415

#### 4. Liste o mês e ano, modalidade, programa de governo e valor de todas as despesas do órgão Ministério da Fazenda no primeiro semestre de 2018.

In [5]:
consulta_questao4 = 'SELECT DESPESA.MES_DESPESA, DESPESA.ANO_DESPESA, DESPESA.MODALIDADE_DESPESA, PROGRAMA.NOME_PROGRAMA_GOVERNO, DESPESA.VALOR_DESPESA TOTAL_DESPESA FROM PROGRAMA_GOVERNO AS PROGRAMA INNER JOIN (SELECT * FROM DESPESA_PUBLICA AS DESPESA INNER JOIN (SELECT * FROM ORGAO WHERE NOME_ORGAO = "Ministério da Fazenda" ) AS ORGAO ON DESPESA.COD_ORGAO = ORGAO.COD_ORGAO) AS DESPESA WHERE DESPESA.MES_DESPESA < 7'
df = pd.read_sql_query(consulta_questao4, conn)
print(df)
csvwriter.writerow([4, df.shape[0], df.shape[1], consulta_questao4])

     MES_DESPESA  ANO_DESPESA       MODALIDADE_DESPESA  \
0              4         2018  Reserva de Contingência   
1              1         2018  Reserva de Contingência   
2              2         2018  Reserva de Contingência   
3              3         2018  Reserva de Contingência   
4              4         2018  Reserva de Contingência   
5              5         2018  Reserva de Contingência   
6              6         2018  Reserva de Contingência   
7              4         2018  Reserva de Contingência   
8              1         2018  Reserva de Contingência   
9              2         2018  Reserva de Contingência   
10             3         2018  Reserva de Contingência   
11             4         2018  Reserva de Contingência   
12             5         2018  Reserva de Contingência   
13             6         2018  Reserva de Contingência   
14             4         2018  Reserva de Contingência   
15             1         2018  Reserva de Contingência   
16            

412

#### 5. Liste o código e nome dos programas de governo que possuíram os 3 maiores valores de despesas totais na função de educação durante o ano de 2018.

In [6]:
consulta_questao5 = 'SELECT DESPESA.COD_PROGRAMA_GOVERNO, DESPESA.NOME_PROGRAMA_GOVERNO FROM  (SELECT * FROM PROGRAMA_GOVERNO AS PROGRAMA INNER JOIN (SELECT * FROM DESPESA_PUBLICA AS DESPESA INNER JOIN (SELECT * FROM DETALHAMENTO_FUNCAO WHERE COD_FUNCAO = (SELECT COD_FUNCAO FROM FUNCAO WHERE NOME_FUNCAO = "Educação" )) AS EDU ON DESPESA.COD_DETALHAMENTO = EDU.COD_DETALHAMENTO) AS DESPESA ON PROGRAMA.COD_PROGRAMA_GOVERNO = DESPESA.COD_PROGRAMA_GOVERNO) AS DESPESA GROUP BY DESPESA.COD_PROGRAMA_GOVERNO ORDER BY SUM(VALOR_DESPESA) DESC LIMIT 3'
df = pd.read_sql_query(consulta_questao5, conn)
print(df)
csvwriter.writerow([5, df.shape[0], df.shape[1], consulta_questao5])

   COD_PROGRAMA_GOVERNO NOME_PROGRAMA_GOVERNO
0                    11                  ENEM
1                    14              PRONATEC
2                    35          MAIS MEDICOS


536

#### 7. Liste os órgãos governamentais e a quantidade de programas de governo distintos envolvidos nas despesas públicas desses órgãos. Nesta consulta, desconsidere valores de despesa negativos ou iguais a zero no perı́odo.

In [7]:
consulta_questao7 = 'SELECT NOME_ORGAO, DESPESAS.NUM_PROGRAMAS FROM ORGAO INNER JOIN (SELECT DISTINCT COUNT(COD_PROGRAMA_GOVERNO) NUM_PROGRAMAS, * FROM DESPESA_PUBLICA WHERE VALOR_DESPESA > 0 GROUP BY COD_ORGAO) AS DESPESAS ON ORGAO.COD_ORGAO = DESPESAS.COD_ORGAO'
df = pd.read_sql_query(consulta_questao7, conn)
print(df)
csvwriter.writerow([7, df.shape[0], df.shape[1], consulta_questao7])

                                      NOME_ORGAO  NUM_PROGRAMAS
0  Ministério da Agricultura, Pecuária e Abastec              4
1                         Ministério da Educação             10
2                            Ministério da Saúde              3
3                           Ministério da Defesa              2
4           Ministério do Desenvolvimento Social              2


250

#### 9. Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui o maior número de órgãos subordinados.

In [8]:
consulta_questao9 = 'SELECT COD_DESPESA, MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, VALOR_DESPESA FROM DESPESA_PUBLICA INNER JOIN (SELECT MAX(QTD_ORGAOS_SUBORDINADOS) QTD, * FROM ORGAO) AS MAIOR_ORGAO ON DESPESA_PUBLICA.COD_ORGAO = MAIOR_ORGAO.COD_ORGAO'
df = pd.read_sql_query(consulta_questao9, conn)
print(df)
csvwriter.writerow([9, df.shape[0], df.shape[1], consulta_questao9])

     COD_DESPESA  MES_DESPESA  ANO_DESPESA  \
0            196           10         2018   
1            197            1         2018   
2            198            2         2018   
3            199            3         2018   
4            200            4         2018   
5            201            5         2018   
6            202            6         2018   
7            203            7         2018   
8            204            9         2018   
9            205           10         2018   
10           206           11         2018   
11           207           12         2018   
12           208            1         2018   
13           209            6         2018   
14           210            7         2018   
15           211            8         2018   
16           212            9         2018   
17           213           11         2018   
18           214           12         2018   
19           215            1         2018   
20           216            2     

245

#### 12. π NOME_FUNCAO, NOME_SUBFUNCAO(FUNCAO ⋈ DETALHAMENTO_FUNCAO ⋈ SUBFUNCAO)

In [9]:
consulta_questao12 = 'SELECT DISTINCT NOME_FUNCAO, NOME_SUBFUNCAO FROM (SELECT * FROM SUBFUNCAO INNER JOIN (SELECT * FROM DETALHAMENTO_FUNCAO INNER JOIN FUNCAO ON DETALHAMENTO_FUNCAO.COD_FUNCAO = FUNCAO.COD_FUNCAO) AS DET ON DET.COD_SUBFUNCAO = SUBFUNCAO.COD_SUBFUNCAO) AS SUB'
df = pd.read_sql_query(consulta_questao12, conn)
print(df)
csvwriter.writerow([12, df.shape[0], df.shape[1], consulta_questao12])

              NOME_FUNCAO                                    NOME_SUBFUNCAO
0     Essencial à justiça                                Comunicação social
1         Defesa nacional                               Administração geral
2         Defesa nacional                                  Defesa terrestre
3       Segurança pública                               Administração geral
4       Segurança pública                                      Policiamento
5       Segurança pública                                      Defesa civil
6       Segurança pública                         Informação e inteligência
7      Assistência social                       Normatização e fiscalização
8      Assistência social                          Tecnologia da informação
9      Assistência social                                Comunicação social
10     Assistência social                           Assistência comunitária
11     Assistência social                            Alimentação e nutrição
12     Assis

264

#### 13. π MODALIDADE_DESPESA, VALOR_DESPESA, NOME_PROGRAMA_GOVERNO(PROGRAMA_GOVERNO ⋈ σ MODALIDADE_DESPESA='Reserva de Contingência'(DESPESA_PUBLICA))

In [10]:
consulta_questao13 = 'SELECT DISTINCT MODALIDADE_DESPESA, VALOR_DESPESA, NOME_PROGRAMA_GOVERNO FROM (SELECT * FROM PROGRAMA_GOVERNO INNER JOIN DESPESA_PUBLICA ON PROGRAMA_GOVERNO.COD_PROGRAMA_GOVERNO = DESPESA_PUBLICA.COD_PROGRAMA_GOVERNO WHERE MODALIDADE_DESPESA = "Reserva de Contingência")'
df = pd.read_sql_query(consulta_questao13, conn)
print(df)
csvwriter.writerow([13, df.shape[0], df.shape[1], consulta_questao13])

         MODALIDADE_DESPESA  VALOR_DESPESA  \
0   Reserva de Contingência           0.00   
1   Reserva de Contingência           0.00   
2   Reserva de Contingência           0.00   
3   Reserva de Contingência        8215.70   
4   Reserva de Contingência           3.42   
5   Reserva de Contingência       81120.13   
6   Reserva de Contingência           0.00   
7   Reserva de Contingência        1497.26   
8   Reserva de Contingência           0.00   
9   Reserva de Contingência           0.00   
10  Reserva de Contingência           0.00   
11  Reserva de Contingência           0.00   
12  Reserva de Contingência           0.00   
13  Reserva de Contingência           0.00   
14  Reserva de Contingência           0.00   
15  Reserva de Contingência        1210.00   
16  Reserva de Contingência           0.00   
17  Reserva de Contingência           0.00   
18  Reserva de Contingência           0.00   
19  Reserva de Contingência         135.36   
20  Reserva de Contingência       

284

#### 14. π VALOR_DESPESA, NOME_ORGAO(DESPESA_PUBLICA ⋈ σ NOME_ORGAO='Ministério da Fazenda'(ORGAO))

In [11]:
consulta_questao14 = 'SELECT DISTINCT VALOR_DESPESA, NOME_ORGAO FROM (SELECT * FROM DESPESA_PUBLICA INNER JOIN ORGAO ON DESPESA_PUBLICA.COD_ORGAO = ORGAO.COD_ORGAO WHERE NOME_ORGAO = "Ministério da Fazenda")'
df = pd.read_sql_query(consulta_questao14, conn)
print(df)
csvwriter.writerow([14, df.shape[0], df.shape[1], consulta_questao14])

   VALOR_DESPESA             NOME_ORGAO
0              0  Ministério da Fazenda


198

#### 15. π NOME_PROGRAMA_GOVERNO, VALOR_DESPESA(DESPESA_PUBLICA ⋈ σ NOME_ORGAO='Ministério da Saúde'(ORGAO) ⋈ PROGRAMA_GOVERNO)

In [12]:
consulta_questao15 = 'SELECT DISTINCT NOME_PROGRAMA_GOVERNO, VALOR_DESPESA FROM PROGRAMA_GOVERNO INNER JOIN (SELECT * FROM DESPESA_PUBLICA INNER JOIN ORGAO ON DESPESA_PUBLICA.COD_ORGAO = ORGAO.COD_ORGAO WHERE NOME_ORGAO = "Ministério da Saúde" ) AS DESPESA ON PROGRAMA_GOVERNO.COD_PROGRAMA_GOVERNO = DESPESA.COD_PROGRAMA_GOVERNO'
df = pd.read_sql_query(consulta_questao15, conn)
print(df)
csvwriter.writerow([15, df.shape[0], df.shape[1], consulta_questao15])

                                NOME_PROGRAMA_GOVERNO  VALOR_DESPESA
0                                    VIVER SEM LIMITE           0.00
1                                      SOS EMERGENCIA           0.00
2                                      MELHOR EM CASA           0.00
3                                                 UBS           0.00
4   UPA (INVESTIMENTO, CUSTEIO E COMPLEXO DE REGUL...           0.00
5                                        REDE CEGONHA           0.00
6                                  CONTROLE DO CANCER           0.00
7                                  CONTROLE DO CANCER       38971.19
8                            CRACK, E POSSIVEL VENCER           0.00
9                                  BRASIL SEM MISERIA           0.00
10  PROGRAMA NACIONAL DE MELHORIA DE ACESSO E QUAL...           0.00
11                                       MAIS MEDICOS           0.00
12                                       MAIS MEDICOS       11000.00
13                                

320

In [13]:
csvfile.close()